In [12]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Fri Mar 24 13:50:39 2017


First let's see if we reproduce an example from [here](http://nbviewer.jupyter.org/github/greglandrum/rdkit_blog/blob/master/notebooks/Trying%20py3Dmol.ipynb):

In [13]:
m = Chem.AddHs(Chem.MolFromSmiles('COc1ccc2[C@H](O)[C@@H](COc2c1)N3CCC(O)(CC3)c4ccc(F)cc4'))
print(m)
AllChem.EmbedMultipleConfs(m,useExpTorsionAnglePrefs=True,useBasicKnowledge=True)
mb = Chem.MolToMolBlock(m)

core = m.GetSubstructMatch(Chem.MolFromSmiles('C1C(O)c2ccccc2OC1'))
print(core)

AllChem.AlignMolConformers(m,atomIds=core)

p = py3Dmol.view(width=400,height=400)
for conf in m.GetConformers():
    mb = Chem.MolToMolBlock(m,confId=conf.GetId())
    p.addModel(mb,'sdf')
p.setStyle({'stick':{}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

(8, 6, 7, 5, 4, 3, 2, 12, 11, 10, 9)


So that works great. Now - let's use sdf files:

In [23]:
mol_inp = Chem.SDMolSupplier("mol_inp.sdf")
mol_out = Chem.SDMolSupplier("mol_out.sdf")
mol_inp, mol_out

(<rdkit.Chem.rdmolfiles.SDMolSupplier at 0x7fc719730998>,
 <rdkit.Chem.rdmolfiles.SDMolSupplier at 0x7fc7197309d0>)

let's see how they look:

* first, *MolToMolBlock* seems to cut hydrogen atoms, but i don't know how else to view a molecule from the sdf file

In [39]:
p1 = py3Dmol.view(width=200,height=200)
m1 = Chem.MolToMolBlock(mol_inp[0])
p1.addModel(m1,'sdf')
p1.setStyle({'stick':{}})
p1.setBackgroundColor('0xeeeeee')
p1.zoomTo()
p1.show()

In [43]:
plist = []
for mo in mol_out:
    p = py3Dmol.view(width=200,height=200)
    m = Chem.MolToMolBlock(mo)
    p.addModel(m,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    plist.append(p)
plist[1].show   

<bound method view.show of <py3Dmol.view object at 0x7fc7193cf908>>

In [33]:
mol1  = open("mol_inp.sdf").read()
p1 = py3Dmol.view(width=200,height=200)
p1.addModel(mol1, 'sdf')
p1.setStyle({'stick':{}})
p1.setBackgroundColor('0xeeeeee')
p1.zoomTo()
p1.show()

TypeError: <rdkit.Chem.rdchem.Mol object at 0x7fc7193c6170> is not JSON serializable

In [16]:
mo = open("mol_out.sdf", "r").readlines()
mo

['\n',
 ' OpenBabel03241713363D\n',
 '\n',
 ' 13 12  0  0  0  0  0  0  0  0999 V2000\n',
 '   -4.4820   -0.1929   -0.0392 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -2.9632   -0.1318   -0.0278 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -4.8449   -0.8813    0.7380 H   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -4.8481   -0.5628   -1.0055 H   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -4.9979    1.2043    0.2047 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -2.5951    0.3397    1.3010 N   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -2.6140    0.5812   -0.7874 H   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -2.5225   -1.1027   -0.2413 H   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -4.6823    1.5824    1.1918 H   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -6.0784    1.2389    0.2350 H   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -4.5971    1.8901   -0.5449 H   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   -2.9636    1.2456    1.4860 H   0  0  0  0  0  0  0  0  0  0  0  0\n',

In [17]:
mol1  = [x for x in mol_inp if x is not None]
mol2  = [x for x in mol_out if x is not None]
mol1, mol2

([<rdkit.Chem.rdchem.Mol at 0x7fc71941f580>],
  <rdkit.Chem.rdchem.Mol at 0x7fc71941fdf0>])

In [18]:
AllChem.AlignMolConformers(mol1, confIds=mol2)

ArgumentError: Python argument types in
    rdkit.Chem.rdMolAlign.AlignMolConformers(list)
did not match C++ signature:
    AlignMolConformers(RDKit::ROMol {lvalue} mol, boost::python::api::object atomIds=[], boost::python::api::object confIds=[], boost::python::api::object weights=[], bool reflect=False, unsigned int maxIters=50, boost::python::api::object RMSlist=None)